In [3]:
config_file = 'delineation/configs/dislocation_matching_disp_and_warp_and var_home.yml'
aug_config_file = 'delineation/configs/aug.yml'
import torch
from delineation.configs.defaults_segmentation import _C as cfg
from delineation.datasets import make_data_loader
from delineation.models import build_model_list
from delineation.utils import settings, cost_volume_helpers
from delineation.utils.settings import evaluate_results

from tqdm import tqdm
%matplotlib inline
import matplotlib.pyplot as plt
import torch.nn.functional as F

import yaml

cfg.merge_from_file(config_file)
with open(aug_config_file, 'r') as ymlfile:
    cfg_aug = yaml.load(aug_config_file)

_device = settings.initialize_cuda_and_logging(cfg)  # '_device' is GLOBAL VAR

train_loader, val_loader = make_data_loader(cfg, cfg_aug)
seg_model, model = build_model_list(cfg, True)

Using cuda
unet
=> loaded checkpoint '/home/dagnyt/EPFL/dislocations/models/dislocations_segmentation/150_segmentor.tar' (epoch 150)
=> loaded checkpoint '/home/dagnyt/EPFL/dislocations/models/dislocations_segmentation/150_segmentor.tar' (epoch 150)
scmnet
=> loaded checkpoint '/home/dagnyt/EPFL/dislocations/models/models_2020_04/dislocations_matching_disp_warp_var_joint_10addaug/150_scmnet_light.tar' (epoch 150)


In [4]:
seg_model.eval()
model.eval()
total_test_loss = 0
pix1_err_m, pix3_err_m, pix5_err_m, epe_m, count = 0, 0, 0, 0, 0

for batch_idx, (l, r, lgt, rgt, dlgt, l_name) in (enumerate(val_loader)):
        indices = cost_volume_helpers.volume_indices(2 * cfg.TRAINING.MAXDISP, len(l),
                                                     cfg.TRAINING.HEIGHT, cfg.TRAINING.WIDTH, _device)

        with torch.no_grad():
            l, r, lgt, rgt, dlgt = l.to(_device), r.to(_device), lgt.to(_device), rgt.to(_device), dlgt.to(_device)

            l_seg, l_segmap = seg_model(l)
            r_seg, r_segmap = seg_model(r)
            if abs(dlgt.max()>32) or abs(dlgt.min()>32):
                continue
            print(l_name)
            dl_scores = model(l_segmap, r_segmap)
            
            dl_ = F.softmax(-dl_scores, 2)
            dl = torch.sum(dl_.mul(indices), 2) - cfg.TRAINING.MAXDISP
            for i in range(0, len(dlgt)):
                pix1_err, pix3_err, pix5_err, epe = evaluate_results(dlgt[i], dl[i], lgt[i])
                print(pix1_err, pix3_err, pix5_err, epe, dlgt.min(), dlgt.max())
                pix1_err_m += pix1_err
                pix3_err_m += pix3_err
                pix5_err_m += pix5_err
                epe_m += epe
                count += 1
            torch.cuda.empty_cache()
            
print('here')
values = pix1_err_m / count, pix3_err_m / count, pix5_err_m / count, epe_m / count
print('Mean per dataset: {}, {}, {}, {}'.format(pix1_err_m / count, pix3_err_m / count, pix5_err_m / count, epe_m / count))

('1116_1116 DPC DF4 71.0 kx DF4-B_LEFT.png',)
0.35424354243542433 0.012076484401207649 0.0 0.8679565 tensor(-4., device='cuda:0') tensor(2., device='cuda:0')
('1123_1123 DPC DF4 71.0 kx DF4-B_LEFT.png',)
0.4160292650482208 0.06484868639840373 0.0 1.1003114 tensor(0., device='cuda:0') tensor(7., device='cuda:0')
('2019-12-02_TiAl_box3n15_zoom_1_2_Aligned_9_of_90001_LEFT.png',)
0.4091642465311391 0.052436269764440144 0.007018393030009681 1.084209 tensor(-4., device='cuda:0') tensor(5., device='cuda:0')
('2019-12-02_TiAl_box3n15_zoom_4_3_Aligned_9_of_90003_LEFT.png',)
0.4355902219363139 0.06794789321325186 0.02203280797684143 1.2023383 tensor(-11., device='cuda:0') tensor(9., device='cuda:0')
('2019-12-02_TiAl_box3n15_zoom_7_5_Aligned_9_of_90007_LEFT.png',)
0.5191446657403119 0.18550254747568318 0.07410838351088467 1.8061295 tensor(-10., device='cuda:0') tensor(12., device='cuda:0')
('2019-12-02_TiAl_box3n15_zoom_7_6_Aligned_9_of_90006_LEFT.png',)
0.3945950167929392 0.03202374443489807 0.